In [154]:
# import h5 reader
import h5py
import numpy as np
import pandas as pd
import time
from os import listdir
from os.path import isfile, join

work_path = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/bar_shearwave/quad4/"
h5_path = work_path + "output/fault/"

In [166]:
# Class RunABatch
class RunABatch:
    # Constructor
    def __init__(self, input_set, work_path, FourierTerms = 16):
        # Initialize data paths and batch parameters
        self.work_path = work_path
        self.input_set = [tuple(i) for i in input_set]
        self.h5_path = work_path + "output/fault/"
        self.FourierTerms = FourierTerms
        
        # Store all cases in the h5_path
        self.existingCasesFile = self.h5_path + "CaseList.csv"
        
        # Flag for whether cases have been run
        self.casesExcuted = False
        
        # Get existing cases
        self.getExistingCasesOfInputSet()
        
        # Get input_set_toRun
        self.input_set_toRun = list(set(self.input_set) - self.existingCases)
        
        # Run cases
        self.runCases(self.input_set_toRun)
        
        # Get obsevations
        self.Observations = self.getObservations(self.input_set)
    
    # Inline function gets [A, B] list
    def getAAndB(self, fileName):
        A_idx = fileName.find('A')
        B_idx = fileName.find('B')
        slash_idx = fileName.find('-')
        
        # Change this part !! Before applying to new name convention
        A = float(fileName[A_idx + 1 : B_idx - 1])
        B = float(fileName[B_idx + 1 : slash_idx])
        return (A, B)
    
    
    # Function get_existing_files_set
    def getExistingCasesOfInputSet(self):
        # Get all .h5 file names as a list
        myPath = self.h5_path
        onlyFiles = [f for f in listdir(myPath) if (isfile(join(myPath, f)) and f[-3 : ] == '.h5')]
        self.existingCases = set([self.getAAndB(f) for f in onlyFiles])
        
    # Function runCases
    def runCases(self, input_set):
        shell_path = self.work_path + "RunJobsJP.sh"
        shellRead = open(shell_path, 'r')
        list_of_lines = shellRead.readlines()
        shellRead.close()

        AA = [ele[0] for ele in input_set]
        BB = [ele[1] for ele in input_set]

        list_of_lines[9] = "AA=" + str(tuple(AA)).replace(',', '') + "\n"
        list_of_lines[10] = "BB=" + str(tuple(BB)).replace(',', '') + "\n"

        shellWrite = open(shell_path, 'w')
        shellWrite.writelines(list_of_lines)
        shellWrite.close()

        # Run the cases
        !source $shell_path
        
        self.casesExcuted = True
        
        # Return from shell
        return
    
    
    # Function getObservations for the input_set
    def getObservations(self, input_set):
        # Initialize Observations
        Observations = []
        
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.h5_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) + "-fault.h5"
            f = h5py.File(h5_file, 'r')

            # Get time
            time = np.array(f['time']).reshape([-1])
            time = time - np.min(time)
            nOfTSteps = time.shape[0]

            # Get Slip rates
            SlipRates = np.array(f['vertex_fields']['slip_rate'])
            Vx = SlipRates[:, :, 0].transpose()
            Vy = SlipRates[:, :, 1].transpose()
            nOfNodes = Vx.shape[0]

            # Find the Fourier coefficients
            FourierTerms = self.FourierTerms
            T = np.max(time)

            # Compute the Fourier terms
            Ks = np.array(range(FourierTerms))
            coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * time)
            VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in Vx], 0)

            # Compute the fourier coefficients
            # print('time.shape: ', time.shape)
            observation = np.trapz(VxcoskPiTt, x=time)

            # Append the result from this file
            Observations.append(observation)
        
        Observations = np.array(Observations)
        return Observations
    
    

In [258]:
# Input example of getting observations
AA = np.linspace(0.002, 0.014, 7)
BB = AA + 0.004

# Get input set
input_set = set([(AA[i], BB[i]) for i in range(len(AA))])

# Test class RunABatch
myBatch = RunABatch(input_set, work_path, FourierTerms = 32)

/home/shengduo/InverseProblems/GPRWorkingField


In [259]:
AA

array([0.002, 0.004, 0.006, 0.008, 0.01 , 0.012, 0.014])

In [260]:
np.array(list(myBatch.input_set))

array([[0.004, 0.008],
       [0.014, 0.018],
       [0.006, 0.01 ],
       [0.01 , 0.014],
       [0.002, 0.006],
       [0.008, 0.012],
       [0.012, 0.016]])

# Sampling process

In [305]:
# Import the sampling packages
import emcee
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
# Set up a GP
myGP = GP_predictor(myBatch.input_set, myBatch.Observations, GPkernel = DotProduct() + WhiteKernel())

# Define the log probability function
def log_prob(u, u_low, u_high, y, GP, si_eta = 1.):
    normal_idx = np.all(np.concatenate([u >= u_low, u <= u_high], axis = 1), axis = 1)
    res = - 0.5 / si_eta ** 2 * np.sum((y - GP.predict(u)) ** 2, axis = 1)
    res[~normal_idx] = -np.inf
    return res

# Use a sample y
y = myBatch.Observations[0]

# Sample from p(u|y)
uLow = np.array([0.002, 0.006])
uLength = np.array([0.01, 0.01])
uHigh = uLow + uLength

ndim, nwalkers = 2, 100
p0 = np.random.uniform(size = [nwalkers, ndim]) * uLength + uSt

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[uLow, uHigh, y, myGP], vectorize = True)
sampler.run_mcmc(p0, 1000)

samples = sampler.get_last_sample().coords
samples.shape

/home/shengduo/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__sigma_0 is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


(100, 2)

In [307]:
np.mean(samples, axis=0)

array([0.0039599 , 0.00771252])

In [310]:

newGP = GP_predictor(np.append(np.array(myBatch.input_set), myInv.U[-2:], axis=0), 
                     np.append(np.array(myBatch.Observations), myInv.O[-2:], axis=0))

# Define the log probability function
def log_prob(u, u_low, u_high, y, GP, si_eta = 1.):
    normal_idx = np.all(np.concatenate([u >= u_low, u <= u_high], axis = 1), axis = 1)
    res = - 0.5 / si_eta ** 2 * np.sum((y - GP.predict(u)) ** 2, axis = 1)
    res[~normal_idx] = -np.inf
    return res

# Use a sample y
y = myBatch.Observations[0]

# Sample from p(u|y)
uLow = np.array([0.002, 0.006])
uLength = np.array([0.01, 0.01])
uHigh = uLow + uLength

ndim, nwalkers = 2, 100
p0 = np.random.uniform(size = [nwalkers, ndim]) * uLength + uSt

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[uLow, uHigh, y, newGP], vectorize = True)
sampler.run_mcmc(p0, 1000)

samples = sampler.get_last_sample().coords
samples.shape

/home/shengduo/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


(100, 2)

In [309]:
np.mean(samples, axis=0)

array([0.00334259, 0.00850793])

# Class BayersianInv

In [370]:
# Define Bayersian Inv that solves a Bayersian inversion problem
class BayersianInv:
    # Constructor
    def __init__(self, u_low, u_high, y, work_path, FourierTerms = 32, 
                 atol = 1.0e-6, si_eta = 0.1, n_samples = 20, MCMCsteps = 1000):
        self.u_low = u_low
        self.u_high = u_high
        self.input_dim = len(u_low)
        
        self.y = y
        self.output_dim = len(y)
        self.work_path = work_path
        self.FourierTerms = FourierTerms
        self.atol = atol
        self.si_eta = si_eta
        self.n_samples = n_samples
        self.MCMCsteps = MCMCsteps
        
        self.GPs = []
        self.U = np.empty([0, self.input_dim])
        self.O = np.empty([0, self.output_dim])
        self.iterations = 0
    
    # Get the accumulative probability function
    def log_prob(self, u, y):
        # Apply hard constraints
        normal_idx = np.all(np.concatenate([u >= self.u_low, u <= self.u_high], axis = 1), axis = 1)
       
        # First prior is uniform distribution
        res = np.ones(len(u))
        res[~normal_idx] = -np.inf
        
        # Add all posteriors in each iteration
        if (np.sum(normal_idx) > 0):
            for i in range(self.iterations):
                res[normal_idx] += -0.5 / self.si_eta ** 2 * (
                    np.sum(
                        (y - self.GPs[i].predict(u[normal_idx])) ** 2, 
                        axis = 1
                    )
                )
        
        # Return the log_probability at the current iteration
        return res
    
    # Initial_run: Explore the parameter space so that Gaussian process can always converge
    def initial_run(self, n_grid_pts = 3):
        return
    
    # Run one iteration
    def runOneIteration(self, n_samples):
        # Sample from the current distribution for u and get the current observations
        sampler = emcee.EnsembleSampler(n_samples, 
                                        self.input_dim, 
                                        self.log_prob, args=[y], 
                                        vectorize = True)
        
        # Initialize uniformly as the starting point
        p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low
        
        # Get the result
        sampler.run_mcmc(p0, self.MCMCsteps)
        self.samples = sampler.get_last_sample().coords
        
        # Check if the standard deviation of the samples is smaller than atol
        if np.std(self.samples) < self.atol:
            # Converging flag True
            return True
        
        else:
            # Run the forward map with the samples
            myBatch = RunABatch(self.samples, self.work_path, FourierTerms = self.FourierTerms)
            self.U = np.append(self.U, self.samples, axis = 0)
            self.O = np.append(self.O, myBatch.Observations, axis = 0)

            # Get a new Gaussian process
            myGP = GP_predictor(self.samples, myBatch.Observations)
            self.GPs.append(myGP)
            self.iterations += 1
            
            # Converging flag False
            return False
    
    # Run function
    def run(self, n_iter_max = 10):
        for i in range(n_iter_max):
            # In each iteration
            print("="*30, " Iteration ", str(self.iterations), " ", "="*30)
            
            # Run one iteration
            start_time = time.time()
            converged = self.runOneIteration(self.n_samples)
            end_time = time.time()
            print("Time cost: ", str(end_time - start_time), " s")
            print("Standard deviatiation of samples: ", np.std(self.samples))
            self.inversed_u = np.mean(self.samples, axis = 0)
            print("Inversed u: ", self.inversed_u)
                
            # Break for pre-achieved convergence
            if converged:
                self.inversed_u = np.mean(self.samples, axis = 0)
                print("Inversion exited with success!")
                print("Inversed u: ", self.inversed_u)
                break
        
        # Print non-convergent info
        self.inversed_u = np.mean(self.samples, axis = 0)
        print("Inversion failed to converge within ", str(self.iterations), " iterations!")
        print("Inversed u: ", self.inversed_u)

# Test an inversion

In [371]:
# Generate test batch
u_low = np.array([0.002, 0.004])
u_high = u_low + 0.01

# Generate input set
AA = np.linspace(0.002, 0.01, 5)
BB = np.linspace(0.004, 0.012, 5)[np.random.permutation(5)]
test_input_set = np.array([[AA[i], BB[i]] for i in range(len(AA))])
fTerms = 32

testBatch = RunABatch(test_input_set, work_path, FourierTerms = fTerms)
print("Test input set: \n", test_input_set)

Running case A0.008_B0.004
Finished in 12 s!

/home/shengduo/InverseProblems/GPRWorkingField
Test input set: 
 [[0.002 0.012]
 [0.004 0.008]
 [0.006 0.006]
 [0.008 0.004]
 [0.01  0.01 ]]


In [375]:
# Run an inversion test:
testCase_idx = 1
y = testBatch.Observations[testCase_idx]
myInv = BayersianInv(u_low, u_high, y, work_path, FourierTerms = fTerms, n_samples = 4)
myInv.run(n_iter_max = 10)

==============================  Iteration  0   ==============================
Running case A0.002692980332901623_B0.00407223411568222
Finished in 12 s!

Running case A0.005174297625034841_B0.008260731108101963
Finished in 12 s!

Running case A0.008502467036649452_B0.004682632019740612
Finished in 13 s!

Running case A0.006030305428158923_B0.00875154342146649
Finished in 12 s!

/home/shengduo/InverseProblems/GPRWorkingField
Time cost:  50.03204655647278  s
Standard deviatiation of samples:  2.1216610566700423e-03
Inversed u:  [0.00560001 0.00644179]
==============================  Iteration  1   ==============================
Running case A5.145529566279162e-03_B0.008247184814070819
Finished in 13 s!

Running case A0.005146046089059924_B0.008243691050468176
Finished in 12 s!

Running case A0.005146094495256704_B0.008245640257331452
Finished in 12 s!

Running case A0.0051417904651657265_B0.008244934901763215
Finished in 13 s!

/home/shengduo/InverseProblems/GPRWorkingField


/home/shengduo/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Time cost:  50.928508281707764  s
Standard deviatiation of samples:  1.550249574069566e-03
Inversed u:  [0.00514487 0.00824536]
==============================  Iteration  2   ==============================
Running case A5.058511263040213e-03_B0.00827684885742331
Finished in 12 s!

Running case A0.005056551667075783_B0.008276346230183347
Finished in 12 s!

Running case A0.005057760365261395_B0.008278107252346598
Finished in 13 s!

Running case A0.005057704971362343_B0.008276936389898994
Finished in 12 s!

/home/shengduo/InverseProblems/GPRWorkingField
Time cost:  51.23710560798645  s
Standard deviatiation of samples:  1.6097139487561108e-03
Inversed u:  [0.00505763 0.00827706]
==============================  Iteration  3   ==============================
Running case A5.046136901981405e-03_B0.008308864150052272
Finished in 13 s!

Running case A0.011999915751656867_B0.006993519972061229
Finished in 12 s!

Running case A0.01199919277873142_B0.006992942995803485
Finished in 13 s!

Running c

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/bar_shearwave/quad4/output/fault/A0.0027011279438323176_B0.004066949415657726-fault.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [264]:
myInv.GPs[1].GP.kernel_

1.05**2 * RBF(length_scale=0.169)

In [269]:
myInv.GPs[9].GP.kernel_

1.42**2 * RBF(length_scale=0.136)

In [221]:
myInv.U

array([[0.00228731, 0.00689832],
       [0.00392835, 0.00934784],
       [0.00840773, 0.0129527 ],
       [0.01103533, 0.01291286],
       [0.00997556, 0.00831904],
       [0.01016095, 0.00954386],
       [0.00513848, 0.00594006],
       [0.00942584, 0.01271636],
       [0.00942076, 0.00678452],
       [0.00935253, 0.00927681],
       [0.00228701, 0.00689931],
       [0.00513804, 0.00593274],
       [0.00228676, 0.0068981 ],
       [0.00831708, 0.01295774],
       [0.00228713, 0.00689779],
       [0.00392835, 0.00934768],
       [0.00392886, 0.00934767],
       [0.00228634, 0.00689834],
       [0.01199574, 0.00727853],
       [0.00392796, 0.00934718],
       [0.00825499, 0.01314345],
       [0.00514722, 0.00651923],
       [0.00825572, 0.01314539],
       [0.00825424, 0.01314462],
       [0.00389768, 0.00937877],
       [0.00389755, 0.00937864],
       [0.00514929, 0.00651952],
       [0.00825467, 0.01314527],
       [0.00514991, 0.00651872],
       [0.00514989, 0.00651862],
       [0.

In [216]:
myInv.O.shape

(100, 96)

In [366]:
# myKernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e2))
# myKernel = DotProduct() + WhiteKernel()

from sklearn.gaussian_process.kernels import Matern

myKernel = 1.0 * Matern(length_scale=1.0e-6, length_scale_bounds=(1e-15, 1e3), nu=1.5)
# myKernel = 1 * RBF(length_scale=1.0e-10, length_scale_bounds=(1e-15, 1e3))


shit = GP_predictor(myInv.U, myInv.O, myKernel)

In [367]:
shit.GP.kernel_

1**2 * Matern(length_scale=1e-06, nu=1.5)

In [368]:
# Define the log probability function
def log_prob(u, u_low, u_high, y, GP, si_eta = 1.):
    normal_idx = np.all(np.concatenate([u >= u_low, u <= u_high], axis = 1), axis = 1)
    res = - 0.5 / si_eta ** 2 * np.sum((y - GP.predict(u)) ** 2, axis = 1)
    res[~normal_idx] = -np.inf
    return res

# Use a sample y
y = myBatch.Observations[0]

# Sample from p(u|y)
uLow = np.array([0.002, 0.006])
uLength = np.array([0.01, 0.01])
uHigh = uLow + uLength

ndim, nwalkers = 2, 100
p0 = np.random.uniform(size = [nwalkers, ndim]) * uLength + uSt

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[uLow, uHigh, y, shit], vectorize = True)
sampler.run_mcmc(p0, 1000)

samples = sampler.get_last_sample().coords
samples.shape

(100, 2)

In [369]:
np.mean(samples, axis = 0)

array([0.00674049, 0.0110458 ])

In [345]:
myInv.U[-10:], myInv.O[-10:]

(array([[0.00377571, 0.00951614],
        [0.0020682 , 0.00708431],
        [0.00377504, 0.00951625],
        [0.00377535, 0.0095161 ],
        [0.00377524, 0.00951633],
        [0.00377529, 0.00951639],
        [0.0020681 , 0.00708436],
        [0.00377541, 0.00951622],
        [0.00377519, 0.00951621],
        [0.00377527, 0.00951647]]),
 array([[-3.60085529e+02,  1.94958264e+01,  1.96437842e+01,
          1.88670169e+01,  1.87778725e+01,  1.80904601e+01,
          1.72596645e+01,  1.65674677e+01,  1.58606433e+01,
          1.41133612e+01,  1.40250724e+01,  1.18417479e+01,
          1.09369004e+01,  9.79981390e+00,  7.51695360e+00,
          6.69149831e+00,  4.79634815e+00,  2.31335324e+00,
          2.12109699e+00, -2.20792679e+00, -1.60127432e+00,
         -6.21755860e+00, -6.34622174e+00, -9.87690222e+00,
         -1.09404858e+01, -1.25180701e+01, -1.26927984e+01,
         -1.07445739e+01, -7.65990549e+00, -2.75989855e+00,
          7.61442397e-01,  3.87137296e+00, -3.59436818e+02

In [236]:
import scipy
from sklearn.utils.optimize import _check_optimize_result
class MyGPR(GaussianProcessRegressor):
    def __init__(self, *args, max_iter=2e06, gtol=1e-06, **kwargs):
        super().__init__(*args, **kwargs)
        self._max_iter = max_iter
        self._gtol = gtol

    def _constrained_optimization(self, obj_func, initial_theta, bounds):
        if self.optimizer == "fmin_l_bfgs_b":
            opt_res = scipy.optimize.minimize(obj_func, initial_theta, method="L-BFGS-B", jac=True, bounds=bounds, options={'maxiter':self._max_iter, 'gtol': self._gtol})
            _check_optimize_result("lbfgs", opt_res)
            theta_opt, func_min = opt_res.x, opt_res.fun
        elif callable(self.optimizer):
            theta_opt, func_min = self.optimizer(obj_func, initial_theta, bounds=bounds)
        else:
            raise ValueError("Unknown optimizer %s." % self.optimizer)
        return theta_opt, func_min

# Gaussian predictor

In [372]:
# Import Gaussian-regression related functions
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Pre-process the data
from sklearn import preprocessing

# function train_GP
class GP_predictor:
    # Constructor
    def __init__(self, 
                 input_set, 
                 observation_set, 
                 GPkernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)), 
                 n_optimizers = 9):
        # Scale input data
        self.input_set = [list(x) for x in input_set]
        self.observation_set = [list(x) for x in observation_set]
        self.input_dimension = len(self.input_set[0])
        self.observation_dimension = len(self.observation_set[0])
        self.trainset_length = len(self.input_set)
        
        self.input_scaler = preprocessing.StandardScaler()
        self.input_scaler.fit(np.array(self.input_set))
        
        # Scale output data
        self.observation_scaler = preprocessing.StandardScaler()
        self.observation_scaler.fit(np.array(observation_set))
        
        # Fit Gaussian process
        self.GP = GaussianProcessRegressor(kernel = GPkernel, n_restarts_optimizer = n_optimizers)
        # self.GP = MyGPR(kernel = GPkernel, n_restarts_optimizer = n_optimizers, max_iter = max_iterations)
        self.GP.fit(self.input_scaler.transform(np.array(self.input_set)), 
                    self.observation_scaler.transform(np.array(self.observation_set)))
        
    # Predict on a new input set
    def predict(self, new_input_set):
        # Predict new observation
        new_observation = self.observation_scaler.inverse_transform(
            self.GP.predict(
                self.input_scaler.transform(np.array(list(new_input_set)).reshape([-1, self.input_dimension]))
            )
        )
        
        return new_observation
        
    

In [69]:
# Generate a set, run cases, get observations

In [ ]:
myGP = GP_predictor(input_set, )